[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_actual_vs_predicted/EMQA_actual_vs_predicted.ipynb)

# EMQA_actual_vs_predicted

Rolling 1-step-ahead RF+GB+LSTM ensemble forecast with bootstrap confidence intervals.
LSTM is trained using PyTorch. Evaluates out-of-sample accuracy using **R²_OOS** (vs naive benchmark), RMSE, MAE, and Direction Accuracy.

**Key Metric:** R²_OOS = 1 - MSE_model / MSE_naive (measures improvement over naive "tomorrow = today" benchmark)

**Output:** `ml_actual_vs_predicted.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
url = 'https://raw.githubusercontent.com/QuantLet/EMQA/main/EMQA_actual_vs_predicted/ro_de_prices_full.csv'
ro = pd.read_csv(url, parse_dates=['date'], index_col='date')
print(f'Loaded {len(ro)} observations')
print(ro.columns.tolist())
ro.head()

In [ ]:
# Build 32-feature set predicting price CHANGES (same as EMQA_feature_importance)
data = ro.copy()
data['target'] = data['ro_price'].diff()  # predict daily price change

# RO lagged levels and changes
for lag in [1, 2, 3, 5, 7, 14, 30]:
    data[f'ro_lag_{lag}'] = data['ro_price'].shift(lag)
    data[f'ro_diff_lag_{lag}'] = data['ro_price'].diff().shift(lag)

# Rolling stats
for w in [7, 14, 30]:
    data[f'ro_ma_{w}'] = data['ro_price'].shift(1).rolling(w).mean()
    data[f'ro_std_{w}'] = data['ro_price'].shift(1).rolling(w).std()

# DE cross-market features
for lag in [1, 7]:
    data[f'de_lag_{lag}'] = data['de_price'].shift(lag)
data['spread_lag1'] = data['ro_price'].shift(1) - data['de_price'].shift(1)

# Temperature
data['ro_temp_lag1'] = data['ro_temp_mean'].shift(1)
data['hdd'] = (18 - data['ro_temp_mean'].shift(1)).clip(lower=0)
data['cdd'] = (data['ro_temp_mean'].shift(1) - 18).clip(lower=0)

# Consumption
data['consumption_lag1'] = data['ro_consumption'].shift(1)
data['consumption_lag7'] = data['ro_consumption'].shift(7)
data['residual_load_lag1'] = data['ro_residual_load'].shift(1)

# Temporal
data['dow'] = data.index.dayofweek
data['month'] = data.index.month
data['weekend'] = (data.index.dayofweek >= 5).astype(int)

data = data.dropna()
exclude = ['target', 'ro_price', 'de_price', 'gas_price',
           'de_temp_mean', 'de_temp_max', 'de_temp_min',
           'ro_temp_mean', 'ro_temp_max', 'ro_temp_min',
           'ro_nuclear', 'ro_hydro', 'ro_coal', 'ro_gas',
           'ro_wind', 'ro_solar', 'ro_consumption', 'ro_residual_load']
feature_cols = [c for c in data.columns if c not in exclude]

print(f"Dataset: {len(data)} rows, {len(feature_cols)} features")
print(f"Features: {feature_cols}")

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- Multivariate LSTM (PyTorch) ---
LOOKBACK = 14
HIDDEN = 64
NUM_LAYERS = 2
EPOCHS = 100
LR = 0.003

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=HIDDEN, num_layers=NUM_LAYERS):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=0.1 if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :]).squeeze(-1)

def make_sequences_mv(features, targets, lookback):
    """Sequence for target[i] uses features[i-L+1:i+1] (includes current step)."""
    X, y = [], []
    for i in range(lookback - 1, len(features)):
        X.append(features[i - lookback + 1:i + 1])
        y.append(targets[i])
    return np.array(X), np.array(y)

def train_lstm_mv(features, targets, lookback=LOOKBACK, epochs=EPOCHS):
    scaler_X = StandardScaler()
    feat_scaled = scaler_X.fit_transform(features)
    X_seq, y_seq = make_sequences_mv(feat_scaled, targets, lookback)
    X_t = torch.FloatTensor(X_seq)
    y_t = torch.FloatTensor(y_seq)
    model = LSTMModel(input_size=features.shape[1], hidden_size=HIDDEN, num_layers=NUM_LAYERS)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_fn = nn.MSELoss()
    model.train()
    for _ in range(epochs):
        optimizer.zero_grad()
        loss = loss_fn(model(X_t), y_t)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    model.eval()
    return model, scaler_X

def predict_lstm_mv(model, scaler_X, recent_features, lookback=LOOKBACK):
    scaled = scaler_X.transform(recent_features[-lookback:])
    X = torch.FloatTensor(scaled).unsqueeze(0)
    with torch.no_grad():
        return model(X).item()

# --- Rolling expanding-window RF+GB+LSTM ensemble forecast ---
init_train = int(len(data) * 0.6)
retrain_every = 30

torch.manual_seed(42)
np.random.seed(42)

rf_model, gb_model = None, None
lstm_model, lstm_scaler = None, None
all_features = data[feature_cols].values
all_targets = data['target'].values  # price changes

ens_preds = []
ci_lo_list, ci_hi_list = [], []
actuals, dates_out = [], []

for i in range(init_train, len(data)):
    step = i - init_train

    # Retrain every 30 steps
    if step % retrain_every == 0:
        X_tr = data[feature_cols].iloc[:i].values
        y_tr = data['target'].iloc[:i].values  # price changes

        rf_model = RandomForestRegressor(
            n_estimators=200, max_depth=10, random_state=42, n_jobs=-1).fit(X_tr, y_tr)
        gb_model = GradientBoostingRegressor(
            n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42).fit(X_tr, y_tr)
        lstm_model, lstm_scaler = train_lstm_mv(all_features[:i], all_targets[:i])

    X_step = data[feature_cols].iloc[i:i+1].values
    prev_price = data['ro_price'].iloc[i - 1]
    actual_price = data['ro_price'].iloc[i]

    # RF prediction: change → level
    rf_chg = rf_model.predict(X_step)[0]
    rf_tree_chg = np.array([t.predict(X_step)[0] for t in rf_model.estimators_])

    # GB prediction: change → level
    gb_chg = gb_model.predict(X_step)[0]
    gb_staged = np.array([p[0] for p in gb_model.staged_predict(X_step)])
    half = max(1, gb_model.n_estimators // 2)
    gb_tree_chg = gb_staged[half:]

    # LSTM prediction: change → level
    lstm_chg = predict_lstm_mv(lstm_model, lstm_scaler, all_features[:i+1])

    # Ensemble: average changes, then convert to level
    ens_chg = (rf_chg + gb_chg + lstm_chg) / 3
    ens_pred = prev_price + ens_chg
    ens_preds.append(ens_pred)

    # CI from tree bootstrap (in levels)
    all_tree_chg = np.concatenate([rf_tree_chg, gb_tree_chg])
    ci_lo_list.append(prev_price + np.percentile(all_tree_chg, 2.5))
    ci_hi_list.append(prev_price + np.percentile(all_tree_chg, 97.5))

    actuals.append(actual_price)
    dates_out.append(data.index[i])

# Convert
ens_preds = np.array(ens_preds)
ci_lo = np.array(ci_lo_list)
ci_hi = np.array(ci_hi_list)
actuals = np.array(actuals)
dates_out = pd.DatetimeIndex(dates_out)

# --- Naive benchmark: tomorrow = today ---
naive_preds = data['ro_price'].iloc[init_train-1:-1].values

# --- Metrics ---
mae = mean_absolute_error(actuals, ens_preds)
rmse = np.sqrt(mean_squared_error(actuals, ens_preds))

mse_model = mean_squared_error(actuals, ens_preds)
mse_naive = mean_squared_error(actuals, naive_preds)

# R²_OOS = 1 - MSE_model / MSE_naive
r2_oos = 1 - mse_model / mse_naive

# Direction accuracy
actual_dir = np.sign(actuals - naive_preds)
pred_dir = np.sign(ens_preds - naive_preds)
dir_acc = np.mean(actual_dir == pred_dir) * 100

# Naive MAE for reference
naive_mae = mean_absolute_error(actuals, naive_preds)
mae_reduction = (1 - mae / naive_mae) * 100

print("=" * 60)
print("   Ensemble (RF+GB+LSTM) vs Naive Forecast")
print("=" * 60)
print(f"{'Naive MAE':<25} {naive_mae:.2f} EUR/MWh")
print(f"{'Ensemble MAE':<25} {mae:.2f} EUR/MWh")
print(f"{'MAE Reduction':<25} {mae_reduction:.0f}%")
print(f"{'RMSE':<25} {rmse:.2f} EUR/MWh")
print(f"{'R²_OOS (vs naive)':<25} {r2_oos*100:.1f}%")
print(f"{'Direction Accuracy':<25} {dir_acc:.1f}%")
print("=" * 60)
if r2_oos > 0:
    print(f">>> Ensemble beats naive by {r2_oos*100:.1f}% R²_OOS")
else:
    print(">>> Ensemble does NOT beat naive")

In [ ]:
# Plot 1: Time series actual vs predicted with CI band
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(dates_out, actuals, color=COLORS['blue'], lw=1.5, label='Actual')
ax.plot(dates_out, ens_preds, color=COLORS['red'], lw=1.5, ls='--',
        label='Ensemble Forecast')
ax.fill_between(dates_out, ci_lo, ci_hi,
                color=COLORS['red'], alpha=0.12, label='95% CI (tree bootstrap)')

# Metrics annotation
textstr = f'R$^2_{{OOS}}$ = {r2_oos*100:.1f}%\nDirection = {dir_acc:.0f}%\nBeats Naive: {"Yes" if r2_oos > 0 else "No"}'
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.set_title('Romanian Electricity: Rolling RF+GB+LSTM Ensemble Forecast', fontsize=14, fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: Scatter actual vs predicted with R2_OOS annotation
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(actuals, ens_preds, color=COLORS['blue'], alpha=0.3, s=15, edgecolors='none')

# Perfect prediction line
lims = [min(actuals.min(), ens_preds.min()), max(actuals.max(), ens_preds.max())]
ax.plot(lims, lims, color=COLORS['red'], ls='--', lw=1.5, label='Perfect Prediction')

# Stats box
textstr = f'R$^2_{{OOS}}$ = {r2_oos*100:.1f}%\nDirection = {dir_acc:.0f}%\nMAE = {mae:.1f} EUR/MWh\nRMSE = {rmse:.1f}'
props = dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.8, edgecolor=COLORS['gray'])
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', bbox=props)

ax.set_xlabel('Actual Price (EUR/MWh)')
ax.set_ylabel('Predicted Price (EUR/MWh)')
ax.set_title('Scatter: Actual vs Ensemble Predicted', fontsize=14, fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

plt.tight_layout()
save_fig(fig, 'ml_actual_vs_predicted.pdf')
plt.show()